In [1]:
import asyncio
import numpy as np

from threading import Thread, Lock
from time import sleep
from enum import Enum

In [2]:
intensity_min = 0.4
service_time_min = 3

sec_multiplier = 0.01
intensity = intensity_min / 60.
service_time = service_time_min * 60

theor_alpha = intensity_min * service_time_min
theor_A = intensity_min / (1 + theor_alpha)
theor_Q = 1 / (1 + theor_alpha)
theor_pn = theor_alpha / (1 + theor_alpha)
theor_p_rej = theor_alpha / (1 + theor_alpha)
theor_p0 = 1 - theor_pn

def wait(sec):
    sleep(sec * sec_multiplier)
    
TOTAL_REQUESTS = 'total_requests'
COMPLETED_REQUESTS = 'completed_requests'
REJECTED_REQUESTS = 'rejected_requests'
CURRENT_TIME = 'current_time'

def get_stats(timer, cmo):
    return {
        CURRENT_TIME: timer.current_time, 
        TOTAL_REQUESTS: cmo.total_requests,
        COMPLETED_REQUESTS: cmo.completed_requests,
        REJECTED_REQUESTS: cmo.rejected_requests,
    }
    

In [3]:
class CMO:
    def __init__(self):
        self.locker = Lock()
        self.init_stats_items()
        self.init_working_items()
        
    
    def init_stats_items(self):
        self.completed_requests = 0
        self.rejected_requests = 0
        self.total_requests = 0
    
    
    def init_working_items(self):
        self.working = False
    
    
    def start(self):
        self.init_stats_items()
        self.working = True
        
        
    def stop(self):
        self.working = False
    
    
    def process_request(self, request):        
        if not self.working:
            return
        
        self.total_requests += 1
        
        if self.locker.locked():
            self.reject_request(request)
            return
        
        self.accept_request(request)
    
    
    def accept_request(self, request):
        self.locker.acquire()
        request.proceed(self.completed_request_callback)
        
        
    def completed_request_callback(self):
        self.completed_requests += 1
        self.locker.release()
    
    
    def reject_request(self, request):
        request.rejected()
        self.rejected_requests += 1
    
    
class Status(Enum):
        NONE = 0
        REJECTED = 1
        COMPLETED = 2    

        
class Request:
    def __init__(self):
        self.status = Status.NONE
        self.service_time = np.random.exponential(service_time)
        
    
    def __proceed(self, callback=None):
        wait(self.service_time)
        self.status = Status.COMPLETED
        if callback:
            callback()
    
        
    def proceed(self, callback=None):
        Thread(target=self.__proceed, args=(callback,)).start()
        
        
    def rejected(self):
        self.status = Status.REJECTED
        
        
class RequestSender():
    def __init__(self, cmo):
        self.working = False
        self.cmo = cmo
            
          
    def start(self, cmo):
        self.working = True
            
            
    def stop(self):
        self.working = False
    
            
    def send_request(self):       
        request = self.create_request()
        if not request:
            return
        self.cmo.process_request(request)    
        
        
    def create_request(self):
        return Request() if np.random.uniform() < intensity else None
    
    
class Timer:
    def __init__(self):
        self.tick_handlers = []
        self.worker: Thread
        self.working = False
        self.current_time = 0
        self.time_step = 1
    
    def add_handler(self, handler):
        self.tick_handlers.append(handler)
    
    def tick(self):
        while self.working:
            for handler in self.tick_handlers:
                handler()
            wait(self.time_step)
            self.current_time += self.time_step
    
    def start(self):
        self.current_time = 0
        self.working = True
        self.worker = Thread(target=self.tick)
        self.worker.start()
        
    def stop(self):
        self.working = False
        
    def clear_handlers():
        self.tick_handlers = []

In [4]:
cmo = CMO()
sender = RequestSender(cmo)
timer = Timer()

sender.start(cmo)
cmo.start()

timer.add_handler(sender.send_request)

timer.start()

In [28]:
stats = get_stats(timer, cmo)

print(f'{stats}\n')

current_time = stats[CURRENT_TIME]
total_requests = stats[TOTAL_REQUESTS]
completed_requests = stats[COMPLETED_REQUESTS]
rejected_requests = stats[REJECTED_REQUESTS]

try:
    emp_intensity = total_requests * 60 / current_time
    emp_alpha = emp_intensity * service_time_min
    emp_p0 = completed_requests/total_requests
    emp_pn = rejected_requests/total_requests
    emp_A = emp_intensity / (1 + emp_alpha)
    emp_Q = 1 / (1 + emp_alpha)
    emp_p_rej = rejected_requests/total_requests
    
    print(f'(Emp, Theor)\nintensity: ({round(emp_intensity, 5)}, {round(intensity_min, 5)})')
    print(f'p0: ({round(emp_p0, 5)}, {round(theor_p0, 5)})')
    print(f'pn: ({round(emp_pn, 5)}, {round(theor_pn, 5)})')
    print(f'A: ({round(emp_A, 5)}, {round(theor_A, 5)})')
    print(f'Q: ({round(emp_Q, 5)}, {round(theor_Q, 5)})')
    print(f'p_rej: ({round(emp_p_rej, 5)}, {round(theor_p_rej, 5)})')
except ZeroDivisionError:
    ...

{'current_time': 2930, 'total_requests': 16, 'completed_requests': 13, 'rejected_requests': 2}

(Emp, Theor)
intensity: (0.32765, 0.4)
p0: (0.8125, 0.45455)
pn: (0.125, 0.54545)
A: (0.16523, 0.18182)
Q: (0.5043, 0.45455)
p_rej: (0.125, 0.54545)


In [29]:
sender.stop()
cmo.stop()
timer.stop()

In [793]:
arr = []

for i in range(1_000_000):
    arr.append(np.random.exponential(service_time))

print(np.mean(arr))

179.77773071112819
